# Examples where risks from automation and COVID-19 intersect

January 2021

In [1]:
%run ../../notebook_preamble_Transitions.ipy

In [2]:
pd.set_option('max_colwidth', 200)

In [3]:
def groupby_isco(occupations, isco_level=3):
    df = occupations.groupby(f'isco_level_{isco_level}').agg({'id': 'count'}).sort_values('id', ascending=False).reset_index()
    df = df.merge(data.isco_titles[['isco', 'isco_title']], left_on = f'isco_level_{isco_level}', right_on = 'isco' , how='left')
    df = df[['isco', 'isco_title', 'id']]
    df = cleanup(df)
    return df

def groupby_sector(occupations, sub_sector=False):
    if sub_sector:
        df = occupations.groupby('sub_sector').agg({'id': 'count'}).sort_values('id', ascending=False).reset_index()
        df = df.merge(data.occ.drop_duplicates('sub_sector')[['sub_sector', 'sub_sector_code']])
        df = df[['sub_sector_code', 'sub_sector', 'id']]
    else:
        df = occupations.groupby('skills_based_sector').agg({'id': 'count'}).sort_values('id', ascending=False).reset_index()
        df = df.merge(data.occ.drop_duplicates('skills_based_sector')[['skills_based_sector', 'skills_based_sector_code']])
        df = df[['skills_based_sector_code', 'skills_based_sector', 'id']]        
    df = cleanup(df)
    return df

def cleanup(df):
    df = df.rename(columns={'id': 'counts'})
    df['proportion'] = np.round(df['counts'] / df['counts'].sum(),3)
    return df    
    

In [4]:
EXPOSURE_THRESHOLD = 0.45

# 1. Sales and services occupations

In [5]:
# Select 'high risk' sales and services workers
occupations = data.occ_report[data.occ_report.skills_based_sector == 'sales & services workers']
occupations = occupations[occupations.risk_category == 'High risk']
len(occupations)

48

In [6]:
# Select 'high risk' sales and services workers that have also a relatively higher COVID-19 exposure
occupations_exposed = occupations[occupations.exposure_score > EXPOSURE_THRESHOLD]
len(occupations_exposed)

40

In [7]:
groupby_isco(occupations_exposed, isco_level=2)

,isco,isco_title,counts,proportion
0,42,Customer services clerks,16,0.400
1,52,Sales workers,15,0.375
2,51,Personal service workers,3,0.075
3,33,Business and administration associate professionals,2,0.050
4,96,Refuse workers and other elementary workers,2,0.050
5,26,"Legal, social and cultural professionals",1,0.025
6,95,Street and related sales and service workers,1,0.025


In [8]:
groupby_sector(occupations_exposed, sub_sector=True)

,sub_sector_code,sub_sector,counts,proportion
0,3.1,"food, cleaning & services workers",18,0.450
1,3.4,retail & sales workers,11,0.275
2,3.2,customer representatives,9,0.225
3,3.3,shop & services managers,1,0.025
4,3.5,wholesale merchants,1,0.025


## 1.1 Transitions

In [9]:
trans = trans_utils.get_transitions(
    origin_ids=occupations_exposed.id.to_list(),
    destination_ids = 'report')
trans = trans[trans.is_viable & trans.is_not_high_risk]
len(trans)

Finding all transitions... Done!
This took 0.70 seconds.


647

In [10]:
# Occupations with transitions
len(trans.origin_id.unique())

39

In [11]:
# Occupations that have transitions to safer occupations
trans_exposure = trans.copy()
trans_exposure = data.add_field_to_occupation(trans_exposure, 'destination_id', 'exposure_score', 'destination')
trans_exposure = data.add_field_to_occupation(trans_exposure, 'origin_id', 'exposure_score', 'origin')
trans_exposure['reduced_exposure'] = trans_exposure['origin_exposure_score'] > trans_exposure['destination_exposure_score']

# Include transitions safer from COVID-19
trans_exposure = trans_exposure[(trans_exposure.destination_exposure_score <= EXPOSURE_THRESHOLD)]

# Remove hospitality sector
stopwords = ['travel', 'tourism', 'hospitality']
for stopword in stopwords:
    trans_exposure = trans_exposure[-trans_exposure.destination_label.str.contains(stopword)]

trans_exposure = trans_exposure.reset_index(drop=True)
print(len(trans_exposure))


169


In [12]:
print(len(trans_exposure.origin_id.unique()))
print(len(trans_exposure.destination_id.unique()))

26
69


## 1.2 Transition destinations

In [13]:
destination_occupations = data.occ_report[data.occ_report.id.isin(trans_exposure.destination_id.to_list())]

### Based on ISCO

In [14]:
groupby_isco(destination_occupations, isco_level=2).head(5)

,isco,isco_title,counts,proportion
0,24,Business and administration professionals,18,0.261
1,33,Business and administration associate professionals,13,0.188
2,12,Administrative and commercial managers,10,0.145
3,13,Production and specialised services managers,10,0.145
4,14,"Hospitality, retail and other services managers",4,0.058


In [15]:
groupby_isco(destination_occupations, isco_level=3).head(5)

,isco,isco_title,counts,proportion
0,243,"Sales, marketing and public relations professionals",15,0.217
1,334,Administrative and specialised secretaries,6,0.087
2,122,"Sales, marketing and development managers",5,0.072
3,134,Professional services managers,5,0.072
4,121,Business services and administration managers,5,0.072


In [16]:
destination_occupations[destination_occupations.isco_level_3==243].preferred_label

139                                                          online community manager
153                                                             ICT presales engineer
430                                                                  network marketer
466                                                                         marketeer
726     technical sales representative in electronic and telecommunications equipment
753                                                          client relations manager
920                                                               ICT account manager
1017                                                                 activism officer
1047                                                      customer experience manager
1214                                                               business developer
1297                                                              fundraising manager
1387                                                  

In [17]:
destination_occupations[destination_occupations.isco_level_3==334].preferred_label

20      legal administrative assistant
574        call centre quality auditor
701             call centre supervisor
728                     office manager
1049         contact centre supervisor
1441               call centre manager
Name: preferred_label, dtype: object

In [18]:
groupby_isco(destination_occupations, isco_level=4).head(5)

,isco,isco_title,counts,proportion
0,2431,Advertising and marketing professionals,9,0.130
1,3341,Office supervisors,5,0.072
2,3339,Business services agents not elsewhere classified,5,0.072
3,1349,Professional services managers not elsewhere classified,5,0.072
4,1221,Sales and marketing managers,4,0.058


### Based on sector

In [19]:
groupby_sector(destination_occupations, sub_sector=True).head(5)

,sub_sector_code,sub_sector,counts,proportion
0,2.4,business managers,21,0.304
1,2.1,"process managers, supervisors & coordinators",17,0.246
2,3.2,customer representatives,12,0.174
3,2.3,"assistants, clerks & legal workers",6,0.087
4,12.1,"ict system engineers, adminstrators & managers",3,0.043


In [20]:
destination_occupations[destination_occupations.sub_sector_code==2.4].preferred_label

102             chief marketing officer
139            online community manager
308                 commercial director
399            business service manager
466                           marketeer
548                    business manager
988                  department manager
1017                   activism officer
1047        customer experience manager
1058                    product manager
1116                ICT product manager
1131           publications coordinator
1214                 business developer
1297                fundraising manager
1311           banking products manager
1327                       talent agent
1448                      brand manager
1449          programme funding manager
1455             advertising specialist
1501    ICT vendor relationship manager
1570            human resources manager
Name: preferred_label, dtype: object

In [21]:
destination_occupations[destination_occupations.sub_sector_code==2.1].preferred_label

36                auction house manager
223                 sport administrator
229                            promoter
358                    venue programmer
421                 court administrator
574         call centre quality auditor
630           ICT documentation manager
701              call centre supervisor
753            client relations manager
849     broadcasting programme director
925      defence administration officer
928                 ICT project manager
1049          contact centre supervisor
1061                     energy manager
1337                  programme manager
1441                call centre manager
1563             contact centre manager
Name: preferred_label, dtype: object

In [22]:
destination_occupations[destination_occupations.sub_sector_code==3.2].preferred_label

107                                                                 call centre agent
142                                                   customer service representative
317                                                             sales account manager
430                                                                  network marketer
632                                                               leaflet distributor
726     technical sales representative in electronic and telecommunications equipment
843                                                          department store manager
920                                                               ICT account manager
1217                                        customer contact centre information clerk
1491                                                     product and services manager
1548                                                                       auctioneer
1564                                                  

### 1.2.1 Most popular destinations

In [23]:
destinations = trans_exposure.groupby('destination_id').agg({'origin_id': 'count'}
                                               ).sort_values('origin_id', ascending=False).reset_index().rename(columns={'origin_id': 'counts'})
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'preferred_label')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'skills_based_sector')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'sub_sector')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'risk')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'prevalence')
destinations.head(10)


,destination_id,counts,preferred_label,skills_based_sector,sub_sector,risk,prevalence
0,2173,22,customer contact centre information clerk,sales & services workers,customer representatives,3.596983,0.569664
1,246,13,customer service representative,sales & services workers,customer representatives,3.596983,0.569664
2,1363,11,client relations manager,business & administration workers,"process managers, supervisors & coordinators",3.596983,0.569664
3,1869,10,customer experience manager,business & administration workers,business managers,3.596983,0.569664
4,563,4,sales account manager,sales & services workers,customer representatives,3.597477,0.544798
5,547,4,commercial director,business & administration workers,business managers,3.525315,0.138942
6,1650,4,ICT account manager,sales & services workers,customer representatives,3.492121,0.259170
7,2816,4,advertising sales agent,sales & services workers,customer representatives,3.532399,0.127752
8,2610,3,advertising specialist,business & administration workers,business managers,3.624351,0.693035
9,1512,3,department store manager,sales & services workers,customer representatives,3.472929,0.233985


In [24]:
destinations.groupby('sub_sector').agg({'counts': 'sum'}).sort_values('counts', ascending=False).head()

,counts
sub_sector,
customer representatives,61
business managers,41
"process managers, supervisors & coordinators",36
"assistants, clerks & legal workers",11
wholesale merchants,5


## 1.3 Skills gaps

In [25]:
gaps = trans_utils.SkillsGaps(trans_exposure)

Time elapsed: 7.51 sec ( 0.044 per transition)


In [28]:
# All destinations
trans_check = trans_exposure;
# # Advertising roles
# trans_check = trans_exposure[trans_exposure.destination_id.isin([2816, 2610])]
len(trans_check)

169

In [30]:
gaps.setup(transition_indices=trans_check.index.to_list(), skills_type='destination')

In [31]:
# Individual skills-level overview
gaps.prevalent_skills_gaps(top_x=95, percentile=True)
# gaps.prevalent_skills_gaps(top_x=20, percentile=False)

,id,preferred_label,level_1,level_2,level_3,counts,prevalence,score,stdev
582,11596,build business relationships,S1,S1.2,S1.2.3,28,0.166,0.805929,0.348005
0,31,guarantee customer satisfaction,A1,A1.12,A1.12.3,62,0.367,0.756484,0.264547
33,710,communicate with customers,S3,S3.4,S3.4.1,57,0.337,0.679877,0.357672
386,7725,perform customer management,S3,S3.4,S3.4.1,22,0.130,0.670636,0.435267
91,1776,manage budgets,S4,S4.3,S4.3.1,26,0.154,0.652538,0.484194
267,5558,identify customer's needs,S1,S1.7,S1.7.2,23,0.136,0.646783,0.448956
393,7794,manage contracts,S1,S1.1,S1.1.1,22,0.130,0.635227,0.491197
141,2778,manage staff,S4,S4.8,S4.8.1,39,0.231,0.609949,0.475152
50,1020,implement marketing strategies,S1,S1.6,S1.6.2,20,0.118,0.530350,0.439355
557,11012,have computer literacy,S5,S5.5,S5.5.0,31,0.183,0.418516,0.500607


In [32]:
# High level overview
gaps.prevalent_cluster_gaps(level='level_1', top_x=75, percentile=True)


,code,title,counts,prevalence,score,stdev,skills
10,S1,"communication, collaboration and creativity",754,4.462,0.235019,0.388647,"create solutions to problems (0.22), build business relationships (0.81), communicate by telephone (0.19)"
13,S4,management skills,662,3.917,0.202860,0.349834,"manage staff (0.61), manage budgets (0.65), ensure customer focus (0.13)"
11,S2,information skills,491,2.905,0.141106,0.304905,"keep records of customer interaction (0.18), collect customer data (0.07), perform market research (0.78)"
6,K4,"business, administration and law",497,2.941,0.100837,0.250274,"characteristics of services (0.23), customer service (0.14), product comprehension (0.14)"


In [33]:
# Mid level overview
gaps.prevalent_cluster_gaps(level='level_2', top_x=80, percentile=True)


,code,title,counts,prevalence,score,stdev,skills
1,A1.12,manage quality,150,0.888,0.528007,0.425649,"guarantee customer satisfaction (0.76), implement customer follow-up (0.35), provide customer follow-up (0.04)"
47,S3.4,providing information and support to the public and clients,147,0.870,0.453048,0.424200,"communicate with customers (0.68), answer incoming calls (0.26), perform customer management (0.67)"
24,S1.1,negotiating,85,0.503,0.305729,0.456017,"manage contracts (0.64), apply conflict management (0.0), handle customer complaints (0.67)"
52,S4.3,allocating and controlling resources,81,0.479,0.295617,0.436548,"manage budgets (0.65), manage accounts (0.01), analyse staff capacity (0.0)"
30,S1.2,liaising and networking,255,1.509,0.256349,0.391483,"build business relationships (0.81), communicate by telephone (0.19), develop professional network (0.14)"
34,S1.6,"promoting, selling and purchasing",113,0.669,0.212549,0.372289,"implement marketing strategies (0.53), implement sales strategies (0.41), maximise sales revenues (0.17)"
39,S2.2,documenting and recording information,124,0.734,0.173073,0.320271,"keep records of customer interaction (0.18), produce sales reports (0.07), keep records on sales (0.0)"
50,S4.1,developing objectives and strategies,252,1.491,0.165476,0.315972,"ensure customer focus (0.13), plan health and safety procedures (0.0), identify new business opportunities (0.21)"
46,S3.3,protecting and enforcing,110,0.651,0.133645,0.288060,"follow company standards (0.09), apply company policies (0.02), ensure information privacy (0.01)"
51,S4.2,"organising, planning and scheduling work and activities",161,0.953,0.121621,0.252169,"supervise work (0.05), prioritise tasks (0.04), fix meetings (0.0)"


In [34]:
# Low level overview
gaps.prevalent_cluster_gaps(level='level_3', top_x=90, percentile=True)


,code,title,counts,prevalence,score,stdev,skills
2,A1.12.3,provide high quality client service,148,0.876,0.528399,0.424550,"guarantee customer satisfaction (0.76), implement customer follow-up (0.35), provide customer follow-up (0.04)"
117,S3.4.1,providing information to the public and clients,129,0.763,0.461140,0.421735,"communicate with customers (0.68), answer incoming calls (0.26), perform customer management (0.67)"
57,S1.2.3,developing professional relationships or networks,120,0.710,0.408467,0.432114,"build business relationships (0.81), develop professional network (0.14), establish customer rapport (0.28)"
73,S1.6.2,"promoting products, services, or programs",64,0.379,0.297484,0.429437,"implement marketing strategies (0.53), implement sales strategies (0.41), apply social media marketing (0.0)"
90,S2.2.5,maintaining operational records,91,0.538,0.218341,0.360142,"keep records of customer interaction (0.18), maintain customer records (0.65), keep records on sales (0.0)"
124,S4.1.2,"developing financial, business or marketing plans",71,0.420,0.177056,0.324855,"strive for company growth (0.13), plan marketing strategy (0.72), plan marketing campaign (0.0)"
28,K4.1.6,wholesale and retail sales,102,0.604,0.175912,0.344198,"characteristics of services (0.23), product comprehension (0.14), sales argumentation (0.39)"
102,S2.7.4,analysing business operations,67,0.396,0.159701,0.342703,"measure customer feedback (0.6), analyse business objectives (0.0), carry out sales analysis (0.0)"
129,S4.2.2,planning and scheduling events and activities,79,0.467,0.138127,0.260851,"fix meetings (0.0), prioritise tasks (0.04), process customer orders (0.22)"
26,K4.1.4,marketing and advertising,130,0.769,0.125223,0.252430,"customer insight (0.18), advertising techniques (0.01), sales strategies (0.6)"


# 2. Clerical admin occupations

In [381]:
# Select 'high risk' sales and services workers
occupations = data.occ_report[data.occ_report.skills_based_sector == 'business & administration workers']
occupations = occupations[occupations.risk_category == 'High risk']
len(occupations)

87

In [382]:
# Select 'high risk' sales and services workers that have also a relatively higher COVID-19 exposure
# occupations_exposed = occupations[occupations.exposure_score > EXPOSURE_THRESHOLD]
occupations_exposed = occupations
len(occupations_exposed)

87

In [383]:
groupby_isco(occupations_exposed, isco_level=2).head(5)

,isco,isco_title,counts,proportion
0,33,Business and administration associate professionals,26,0.299
1,43,Numerical and material recording clerks,22,0.253
2,24,Business and administration professionals,12,0.138
3,13,Production and specialised services managers,8,0.092
4,12,Administrative and commercial managers,4,0.046


In [384]:
groupby_isco(occupations_exposed, isco_level=3).head(5)

,isco,isco_title,counts,proportion
0,432,Material-recording and transport clerks,13,0.149
1,331,Financial and mathematical associate professionals,11,0.126
2,431,Numerical clerks,9,0.103
3,242,Administration professionals,7,0.080
4,332,Sales and purchasing agents and brokers,5,0.057


In [385]:
groupby_sector(occupations_exposed, sub_sector=True)

,sub_sector_code,sub_sector,counts,proportion
0,2.3,"assistants, clerks & legal workers",30,0.345
1,2.2,accounting & financial services workers,27,0.310
2,2.1,"process managers, supervisors & coordinators",19,0.218
3,2.4,business managers,6,0.069
4,2.5,public officials,5,0.057


In [386]:
occupations_exposed[occupations_exposed.sub_sector_code==2.3].preferred_label

31            administrative assistant
48              recruitment consultant
83      telephone switchboard operator
211                     auditing clerk
217                 relocation officer
269              middle office analyst
283                executive assistant
303          social security inspector
325                        bank teller
382                  licensing officer
439                   employment agent
490             tax compliance officer
598                          secretary
608            sales support assistant
708                       office clerk
713               occupational analyst
942               management assistant
1036                        file clerk
1119      court administrative officer
1151                     payroll clerk
1186            back office specialist
1218          foreign exchange cashier
1233                     billing clerk
1274              accounting assistant
1313                property assistant
1320                     

## 2.1 Transitions

In [387]:
# Select 'high risk' sales and services workers that have also a relatively higher COVID-19 exposure
occupations_exposed = occupations[occupations.exposure_score > EXPOSURE_THRESHOLD]
# occupations_exposed = occupations
len(occupations_exposed)

12

In [394]:
occupations_exposed

,id,concept_type,concept_uri,preferred_label,isco_level_1,isco_level_2,isco_level_3,isco_level_4,is_top_level,job_zone,...,onet_occupation,level_1,level_2,skills_based_sector_code,sub_sector_code,skills_based_sector,sub_sector,remote_labor_index,physical_proximity,exposure_score
83,148,Occupation,http://data.europa.eu/esco/occupation/0bf68df5-e808-4c27-abf2-6f85440f2d7f,telephone switchboard operator,4,42,422,4223,True,2.0,...,"switchboard operators, including answering service",1,8,2,2.3,business & administration workers,"assistants, clerks & legal workers",0.411765,0.6325,0.609966
222,399,Occupation,http://data.europa.eu/esco/occupation/20b9f370-6b56-4679-8c06-c23b866a5bd0,labour relations officer,2,24,242,2423,True,4.0,...,labor relations specialists,1,10,2,2.5,business & administration workers,public officials,0.538462,0.5125,0.486352
381,693,Occupation,http://data.europa.eu/esco/occupation/39a59c5c-4338-4480-bdc8-96328a491907,attraction operator,9,96,962,9629,True,1.0,...,amusement and recreation attendants,1,6,2,2.1,business & administration workers,"process managers, supervisors & coordinators",0.375000,0.8350,0.722409
382,695,Occupation,http://data.europa.eu/esco/occupation/39c89133-bd2a-45db-b3eb-529b94daeeaf,licensing officer,3,33,335,3354,True,3.0,...,licensing examiners and inspectors,1,8,2,2.3,business & administration workers,"assistants, clerks & legal workers",0.641026,0.6950,0.499487
598,1094,Occupation,http://data.europa.eu/esco/occupation/5ac377d2-e1d0-4911-a299-b4ef2782a0ba,secretary,4,41,412,4120,True,2.0,...,"secretaries and administrative assistants, except legal, medical, and executive",1,8,2,2.3,business & administration workers,"assistants, clerks & legal workers",0.448276,0.4250,0.484234
708,1290,Occupation,http://data.europa.eu/esco/occupation/6c999fc7-c6b7-4ef3-a4b9-af124a1783a2,office clerk,4,41,411,4110,True,2.0,...,"office clerks, general",1,8,2,2.3,business & administration workers,"assistants, clerks & legal workers",0.416667,0.6425,0.612202
890,1602,Occupation,http://data.europa.eu/esco/occupation/857ed740-f4ae-43d0-8295-d6181f1d7eed,funeral services director,5,51,516,5163,True,3.0,...,"morticians, undertakers, and funeral directors",1,6,2,2.1,business & administration workers,"process managers, supervisors & coordinators",0.437500,0.7200,0.636396
955,1709,Occupation,http://data.europa.eu/esco/occupation/8ec8df02-e9dd-43b7-b416-5846ae0414ab,real estate agent,3,33,333,3334,True,3.0,...,real estate sales agents,1,7,2,2.2,business & administration workers,accounting & financial services workers,0.700000,0.7100,0.461519
1036,1851,Occupation,http://data.europa.eu/esco/occupation/9ab0f0db-1128-4050-becd-b09db5eabdaa,file clerk,4,44,441,4415,True,2.0,...,file clerks,1,8,2,2.3,business & administration workers,"assistants, clerks & legal workers",0.500000,0.4675,0.483477
1119,1990,Occupation,http://data.europa.eu/esco/occupation/a7f63810-a21a-4e7e-8aa2-6637d14ea3c1,court administrative officer,3,34,341,3411,True,2.0,...,court clerks,1,8,2,2.3,business & administration workers,"assistants, clerks & legal workers",0.571429,0.6225,0.516513


In [395]:
trans = trans_utils.get_transitions(
    origin_ids=occupations_exposed.id.to_list(),
    destination_ids = 'report')
trans = trans[trans.is_viable & trans.is_not_high_risk]
len(trans)

Finding all transitions... Done!
This took 0.17 seconds.


203

In [396]:
# Occupations with transitions
len(trans.origin_id.unique())

12

In [397]:
# Occupations that have transitions to safer occupations
trans_exposure = trans.copy()
trans_exposure = data.add_field_to_occupation(trans_exposure, 'destination_id', 'exposure_score', 'destination')
trans_exposure = data.add_field_to_occupation(trans_exposure, 'origin_id', 'exposure_score', 'origin')
trans_exposure['reduced_exposure'] = trans_exposure['origin_exposure_score'] > trans_exposure['destination_exposure_score']

# Include transitions safer from COVID-19
trans_exposure = trans_exposure[(trans_exposure.destination_exposure_score <= EXPOSURE_THRESHOLD)]

# Remove hospitality sector
stopwords = ['travel', 'tourism', 'hospitality']
for stopword in stopwords:
    trans_exposure = trans_exposure[-trans_exposure.destination_label.str.contains(stopword)]

trans_exposure = trans_exposure.reset_index(drop=True)
print(len(trans_exposure))


127


In [398]:
print(len(trans_exposure.origin_id.unique()))
print(len(trans_exposure.destination_id.unique()))

11
88


## 2.2 Transition destinations

In [399]:
destination_occupations = data.occ_report[data.occ_report.id.isin(trans_exposure.destination_id.to_list())]

### 2.2.1 Based on ISCO

In [400]:
groupby_isco(destination_occupations, isco_level=2).head(5)

,isco,isco_title,counts,proportion
0,24,Business and administration professionals,22,0.250
1,33,Business and administration associate professionals,21,0.239
2,11,"Chief executives, senior officials and legislators",11,0.125
3,13,Production and specialised services managers,10,0.114
4,12,Administrative and commercial managers,8,0.091


In [401]:
groupby_isco(destination_occupations, isco_level=3).head(5)

,isco,isco_title,counts,proportion
0,243,"Sales, marketing and public relations professionals",10,0.114
1,111,Legislators and senior officials,9,0.102
2,242,Administration professionals,9,0.102
3,333,Business services agents,8,0.091
4,331,Financial and mathematical associate professionals,7,0.080


In [402]:
destination_occupations[destination_occupations.isco_level_3==243].preferred_label

753                   client relations manager
875                             election agent
920                        ICT account manager
962     intercultural communication consultant
1051                 public affairs consultant
1178                    advertising copywriter
1279                  public relations officer
1297                       fundraising manager
1455                    advertising specialist
1574                     political party agent
Name: preferred_label, dtype: object

In [403]:
destination_occupations[destination_occupations.isco_level_3==334].preferred_label

20      legal administrative assistant
565                    civil registrar
728                     office manager
1049         contact centre supervisor
Name: preferred_label, dtype: object

### 2.2.2 Based on sector

In [404]:
groupby_sector(destination_occupations, sub_sector=True).head(5)

,sub_sector_code,sub_sector,counts,proportion
0,2.5,public officials,20,0.227
1,2.1,"process managers, supervisors & coordinators",16,0.182
2,2.2,accounting & financial services workers,14,0.159
3,2.4,business managers,11,0.125
4,2.3,"assistants, clerks & legal workers",10,0.114


In [405]:
destination_occupations[destination_occupations.sub_sector_code==2.1][['preferred_label','isco_level_4']]

,preferred_label,isco_level_4
36,auction house manager,1221
141,waste management supervisor,3122
223,sport administrator,1349
421,court administrator,1349
426,production supervisor,3122
610,secretary general,1112
630,ICT documentation manager,1330
753,client relations manager,2431
849,broadcasting programme director,3435
925,defence administration officer,1219


In [406]:
destination_occupations[destination_occupations.sub_sector_code==2.2][['preferred_label','isco_level_4']]

,preferred_label,isco_level_4
13,insurance claims handler,3315
111,real estate manager,3334
232,financial planner,2412
239,foreclosure specialist,3315
307,insurance collector,4214
620,bank account manager,3312
756,personal trust officer,2412
953,loan officer,3312
1135,mortgage loan underwriter,3312
1223,property acquisitions manager,3334


In [415]:
destinations = trans_exposure.groupby('destination_id').agg({'origin_id': 'count'}
                                               ).sort_values('origin_id', ascending=False).reset_index().rename(columns={'origin_id': 'counts'})
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'preferred_label')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'skills_based_sector')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'sub_sector')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'risk')
destinations = data.add_field_to_occupation(destinations, 'destination_id', 'prevalence')
destinations.head(10)


,destination_id,counts,preferred_label,skills_based_sector,sub_sector,risk,prevalence
0,35,6,legal administrative assistant,business & administration workers,"assistants, clerks & legal workers",3.664614,0.508410
1,2940,5,civil service administrative officer,business & administration workers,"assistants, clerks & legal workers",3.330145,0.614912
2,1319,4,office manager,business & administration workers,"assistants, clerks & legal workers",3.575837,0.636072
3,2173,4,customer contact centre information clerk,sales & services workers,customer representatives,3.596983,0.569664
4,1663,4,defence administration officer,business & administration workers,"process managers, supervisors & coordinators",3.330145,0.614912
5,765,3,court administrator,business & administration workers,"process managers, supervisors & coordinators",3.330145,0.614912
6,1043,2,civil registrar,business & administration workers,"assistants, clerks & legal workers",3.664614,0.508410
7,2590,2,housing manager,business & administration workers,accounting & financial services workers,3.415209,0.551901
8,2370,2,talent agent,business & administration workers,business managers,3.412754,0.457125
9,1129,2,bank account manager,business & administration workers,accounting & financial services workers,3.484457,0.427744


In [416]:
destinations.groupby('sub_sector').agg({'counts': 'sum'}).sort_values('counts', ascending=False).head()

,counts
sub_sector,
"assistants, clerks & legal workers",27
"process managers, supervisors & coordinators",22
public officials,20
accounting & financial services workers,19
business managers,17


In [417]:
data.occ[data.occ.preferred_label.str.contains('hair')].id

,id,concept_type,concept_uri,preferred_label,isco_level_1,isco_level_2,isco_level_3,isco_level_4,is_top_level,job_zone,...,onet_occupation,level_1,level_2,skills_based_sector_code,sub_sector_code,skills_based_sector,sub_sector,remote_labor_index,physical_proximity,exposure_score
120,120,Occupation,http://data.europa.eu/esco/occupation/099c6bb0-22d3-4c5d-8bf5-70910af381ef,hairdresser,5,51,514,5141,True,3.0,...,"hairdressers, hairstylists, and cosmetologists",2,11,3,3.1,sales & services workers,"food, cleaning & services workers",0.117647,0.9225,0.902203
1026,1026,Occupation,http://data.europa.eu/esco/occupation/55f6380c-7511-4c33-8fc4-c74f10f524fa,hairdresser assistant,5,51,514,5141,False,3.0,...,"hairdressers, hairstylists, and cosmetologists",2,11,3,3.1,sales & services workers,"food, cleaning & services workers",0.117647,0.9225,0.902203
1125,1125,Occupation,http://data.europa.eu/esco/occupation/5dd01019-3999-4d90-bc8a-7a04b0b208b0,make-up and hair designer,5,51,514,5142,True,3.0,...,"makeup artists, theatrical and performance",5,23,6,6.1,arts & media workers,art & media technicians,0.454545,0.8575,0.683906
1362,1362,Occupation,http://data.europa.eu/esco/occupation/72413d91-f212-4ad6-bf95-8b78a37c041a,hair removal technician,5,51,514,5142,True,3.0,...,skincare specialists,2,11,3,3.1,sales & services workers,"food, cleaning & services workers",0.100000,0.9875,0.942735
1570,1570,Occupation,http://data.europa.eu/esco/occupation/82dea303-26e1-4bbc-a07c-cff3cac6b50e,hair stylist,5,51,514,5141,False,3.0,...,"hairdressers, hairstylists, and cosmetologists",5,25,6,6.3,arts & media workers,creative managers & graphic designers,0.117647,0.9225,0.902203
1754,1754,Occupation,http://data.europa.eu/esco/occupation/93f61216-b9c8-4051-bb1e-32839c1566c1,hairdressing vocational teacher,2,23,232,2320,False,3.0,...,"vocational education teachers, postsecondary",6,28,7,7.1,education workers,instructors & vocational teachers,0.733333,0.7575,0.449444
2236,2236,Occupation,http://data.europa.eu/esco/occupation/bd206ee9-61d6-45de-a369-0c71b8429efa,wig and hairpiece maker,7,75,753,7531,True,3.0,...,"hairdressers, hairstylists, and cosmetologists",5,23,6,6.1,arts & media workers,art & media technicians,0.117647,0.9225,0.902203
2247,2247,Occupation,http://data.europa.eu/esco/occupation/bdc74965-1d1a-4f8e-a490-9cf71f494e02,dental chairside assistant,3,32,325,3251,True,3.0,...,dental assistants,10,42,11,11.2,healthcare workers,healthcare technicians & scientists,0.285714,0.9850,0.838792
2828,2828,Occupation,http://data.europa.eu/esco/occupation/f60dc59b-7681-45d4-8114-b3a646e02ab2,performance hairdresser,5,51,514,5141,False,3.0,...,"hairdressers, hairstylists, and cosmetologists",5,23,6,6.1,arts & media workers,art & media technicians,0.117647,0.9225,0.902203


## 2.3 Skills gaps

In [409]:
gaps = trans_utils.SkillsGaps(trans_exposure)

Time elapsed: 3.78 sec ( 0.030 per transition)


In [410]:
gaps.setup(transition_indices=trans_exposure.index.to_list(), skills_type='destination')

In [411]:
# Individual skills-level overview
gaps.prevalent_skills_gaps(top_x=95, percentile=True)

,id,preferred_label,level_1,level_2,level_3,counts,prevalence,score,stdev
272,5035,maintain relationships with government agencies,S1,S1.2,S1.2.3,9,0.071,0.998000,0.000000
353,6062,obtain financial information,S2,S2.4,S2.4.1,10,0.079,0.998000,0.000000
124,2369,government policy implementation,K3,K3.1,K3.1.2,14,0.110,0.998000,0.000000
463,7794,manage contracts,S1,S1.1,S1.1.1,10,0.079,0.799600,0.418264
153,2778,manage staff,S4,S4.8,S4.8.1,31,0.244,0.752097,0.398183
249,4633,create a financial plan,S4,S4.1,S4.1.2,11,0.087,0.715455,0.445084
563,9481,property law,K4,K4.2,K4.2.0,10,0.079,0.698600,0.482080
690,11677,communicate by telephone,S1,S1.2,S1.2.0,10,0.079,0.598800,0.515365
200,3595,keep task records,S2,S2.2,S2.2.5,9,0.071,0.562333,0.516677
376,6387,manage administrative systems,S4,S4.4,S4.4.3,19,0.150,0.551684,0.378423


In [412]:
# High level overview
gaps.prevalent_cluster_gaps('level_3', top_x=90, percentile=True)


,code,title,counts,prevalence,score,stdev,skills
151,S4.8.1,supervising a team or group,44,0.346,0.554136,0.460079,"manage staff (0.75), supervise staff (0.06), manage sales teams (0.01)"
60,S1.2.3,developing professional relationships or networks,81,0.638,0.474407,0.473549,"develop professional network (0.21), build business relationships (0.1), maintain relationships with government agencies (1.0)"
129,S3.4.1,providing information to the public and clients,39,0.307,0.412692,0.466663,"respond to enquiries (0.39), communicate with customers (0.55), answer incoming calls (0.33)"
147,S4.4.3,performing general clerical and administrative tasks,52,0.409,0.393423,0.428934,"manage administrative systems (0.55), perform clerical duties (0.19), perform office routine activities (0.34)"
23,K3.1.2,political sciences and civics,37,0.291,0.388541,0.483143,"government policy implementation (1.0), diplomatic principles (0.0), government representation (0.04)"
142,S4.3.1,managing budgets or finances,59,0.465,0.313983,0.446446,"manage budgets (0.52), enforce financial policies (0.28), manage accounts (0.42)"
108,S2.7.3,analysing financial and economic data,40,0.315,0.268450,0.365062,"analyse financial performance of a company (0.6), assess financial viability (0.13), analyse external factors of companies (0.13)"
134,S4.1.2,"developing financial, business or marketing plans",37,0.291,0.242405,0.403122,"create a financial plan (0.72), strive for company growth (0.01), develop public relations strategies (0.01)"
29,K4.1.2,"finance, banking and insurance",70,0.551,0.203657,0.381013,"budgetary principles (0.0), financial analysis (0.6), financial statements (0.0)"
61,S1.2.4,advocating for individual or community needs,42,0.331,0.157929,0.348715,"protect client interests (0.41), present arguments persuasively (0.0), apply knowledge of human behaviour (0.0)"


In [413]:
# Mid level overview
gaps.prevalent_cluster_gaps('level_2', top_x=80, percentile=True)



,code,title,counts,prevalence,score,stdev,skills
62,S4.8,supervising people,70,0.551,0.375043,0.447722,"manage staff (0.75), delegate activities (0.04), supervise staff (0.06)"
58,S4.4,performing administrative activities,66,0.520,0.344167,0.413798,"manage administrative systems (0.55), perform clerical duties (0.19), perform office routine activities (0.34)"
28,S1.1,negotiating,53,0.417,0.329774,0.465184,"manage contracts (0.8), apply conflict management (0.33), handle conflicts (0.67)"
34,S1.2,liaising and networking,231,1.819,0.281377,0.424293,"develop professional network (0.21), liaise with managers (0.03), protect client interests (0.41)"
43,S2.2,documenting and recording information,76,0.598,0.248421,0.404826,"keep task records (0.56), keep records of customer interaction (0.25), compile legal documents (0.22)"
57,S4.3,allocating and controlling resources,96,0.756,0.239396,0.392131,"manage budgets (0.52), enforce financial policies (0.28), manage accounts (0.42)"
37,S1.5,advising and consulting,87,0.685,0.196345,0.341909,"advise on financial matters (0.34), advise on public relations (0.15), advise on legislative acts (0.0)"
18,K4.1,business and administration,252,1.984,0.167679,0.346923,"corporate social responsibility (0.0), budgetary principles (0.0), customer service (0.5)"
47,S2.7,analysing and evaluating information and data,103,0.811,0.164621,0.318656,"analyse financial performance of a company (0.6), analyse financial risk (0.05), assess financial viability (0.13)"
19,K4.2,law,101,0.795,0.136792,0.336287,"property law (0.7), legislation procedure (0.0), copyright legislation (0.0)"


In [414]:
# Low level overview
gaps.prevalent_cluster_gaps('level_1', top_x=10, percentile=True)


,code,title,counts,prevalence,score,stdev,skills
6,K3,"social sciences, journalism and information",54,0.425,0.268574,0.436520,"government policy implementation (1.0), diplomatic principles (0.0), government representation (0.04)"
15,S4,management skills,494,3.890,0.222905,0.375110,"manage staff (0.75), manage budgets (0.52), manage administrative systems (0.55)"
12,S1,"communication, collaboration and creativity",590,4.646,0.218307,0.378074,"liaise with managers (0.03), develop professional network (0.21), protect client interests (0.41)"
13,S2,information skills,332,2.614,0.204985,0.374060,"obtain financial information (1.0), keep records of customer interaction (0.25), keep task records (0.56)"
9,K6,information and communication technologies (icts),13,0.102,0.191615,0.361308,"office software (0.24), industrial software (0.02), information confidentiality (0.5)"
14,S3,assisting and caring,140,1.102,0.186557,0.340503,"follow company standards (0.12), respond to enquiries (0.39), communicate with customers (0.55)"
7,K4,"business, administration and law",353,2.780,0.158841,0.343722,"corporate social responsibility (0.0), budgetary principles (0.0), customer service (0.5)"
16,S5,working with computers,16,0.126,0.114250,0.244375,"perform data analysis (0.0), manage office facility systems (0.36), use customer relationship management software (0.12)"
0,A1,attitudes,49,0.386,0.112837,0.242993,"guarantee customer satisfaction (0.31), meet deadlines for preparing legal cases (0.04), implement customer follow-up (0.37)"
2,K0,generic programmes and qualifications,9,0.071,0.109111,0.309384,"communication principles (0.01), communication (0.47)"


# Check COVID and automation risk exposure

In [474]:
df = data.occ_report.groupby('isco_level_3').mean().reset_index().merge(data.isco_titles[['isco', 'isco_title']], left_on='isco_level_2', right_on='isco')
dff = df[['risk', 'exposure_score', 'isco_title']]

In [477]:
px.scatter(dff, x='risk', y='exposure_score', hover_data=['isco_title'])